# Experiments on the Reproducibility of Results with LinTS
## Imports and Custom Class

In [1]:
# Run this cell for every experiment
import pandas as pd
import numpy as np
import platform
import pickle
from sklearn.preprocessing import StandardScaler

from mabwiser.mab import MAB
from mabwiser.linear import _RidgeRegression, _Linear

class LinTSExample(_RidgeRegression):
    def predict(self, x):
        beta_sampled = rng.multivariate_normal(x, self.A_inv)
        return np.dot(x, beta_sampled)
    
class LinearExample(_Linear):
    factory = {"ts": LinTSExample}

    def __init__(self, rng, arms, n_jobs=1, backend=None, l2_lambda=1, alpha=1, regression='ts', arm_to_scaler = None):
        super().__init__(rng, arms, n_jobs, backend, l2_lambda, alpha, regression)
       
        self.l2_lambda = l2_lambda
        self.alpha = alpha
        self.regression = regression

        # Create ridge regression model for each arm
        self.num_features = None

        if arm_to_scaler is None:
            arm_to_scaler = dict((arm, None) for arm in arms)

        self.arm_to_model = dict((arm, LinearExample.factory.get(regression)(rng, l2_lambda,
                                                                       alpha, arm_to_scaler[arm])) for arm in arms)
 

# Mac OS

In [2]:
platform.platform()

'macOS-10.15.7-x86_64-i386-64bit'

In [3]:
print(np.__version__)

1.18.5


In [4]:
users = pd.read_csv('users.csv')
responses = pd.read_csv('responses.csv')

train = users[users['set']=='train']
test = users[users['set']=='test']

train = train.merge(responses, how='left', on='user id')
context_features = [c for c in users.columns if c not in ['user id', 'set']]

decisions = MAB._convert_array(train['item id'])
rewards = MAB._convert_array(train['rated'])
contexts = MAB._convert_matrix(train[context_features]).astype('float')

test_contexts = MAB._convert_matrix(test[context_features]).astype('float')

rng = np.random.RandomState(seed=11)
item_ids = list(responses['item id'].unique())
mab = LinearExample(rng=rng, arms=item_ids, l2_lambda=1, alpha=1, regression='ts', n_jobs=1, backend=None)
mab.arm_to_model[1]

In [5]:
scaler = pickle.load(open('scaler.pkl', 'rb'))
contexts = scaler.fit_transform(contexts)
test_contexts = scaler.transform(test_contexts)

mab.fit(decisions, rewards, contexts)
mac_expectations = mab.predict_expectations(test_contexts)

mac_expectations[0][1]

13.999455973138591

In [6]:
mac_expectations[0][2]

13.619175409413963

In [7]:
pickle.dump(mab, open('mac_mab3.pkl', 'wb'))
pickle.dump(mac_expectations, open('mac_expectations3.pkl', 'wb'))

# Mac OS 2

In [2]:
platform.platform()

'Darwin-19.5.0-x86_64-i386-64bit'

In [3]:
print(np.__version__)

1.18.1


In [4]:
users = pd.read_csv('users.csv')
responses = pd.read_csv('responses.csv')

train = users[users['set']=='train']
test = users[users['set']=='test']

train = train.merge(responses, how='left', on='user id')
context_features = [c for c in users.columns if c not in ['user id', 'set']]

decisions = MAB._convert_array(train['item id'])
rewards = MAB._convert_array(train['rated'])
contexts = MAB._convert_matrix(train[context_features]).astype('float')

test_contexts = MAB._convert_matrix(test[context_features]).astype('float')

rng = np.random.RandomState(seed=11)
item_ids = list(responses['item id'].unique())
mab = LinearExample(rng=rng, arms=item_ids, l2_lambda=1, alpha=1, regression='ts', n_jobs=1, backend=None)
mab.arm_to_model[1]

In [5]:
scaler = pickle.load(open('scaler.pkl', 'rb'))
contexts = scaler.fit_transform(contexts)
test_contexts = scaler.transform(test_contexts)

mab.fit(decisions, rewards, contexts)
mac_expectations = mab.predict_expectations(test_contexts)

mac_expectations[0][1]

/Users/emily/anaconda3/lib/python3.7/site-packages/sklearn/base.py:318: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 0.22.1. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


13.999455973138692

In [6]:
mac_expectations[0][2]

13.619175409413883

In [7]:
pickle.dump(mab, open('mac_mab4.pkl', 'wb'))
pickle.dump(mac_expectations, open('mac_expectations4.pkl', 'wb'))

# Red Hat Linux

In [2]:
platform.platform()

'Linux-3.10.0-1160.15.2.el7.x86_64-x86_64-with-glibc2.10'

In [3]:
print(np.__version__)

1.19.2


In [4]:
users = pd.read_csv('users.csv')
responses = pd.read_csv('responses.csv')

train = users[users['set']=='train']
test = users[users['set']=='test']

train = train.merge(responses, how='left', on='user id')
context_features = [c for c in users.columns if c not in ['user id', 'set']]

decisions = MAB._convert_array(train['item id'])
rewards = MAB._convert_array(train['rated'])
contexts = MAB._convert_matrix(train[context_features]).astype('float')

test_contexts = MAB._convert_matrix(test[context_features]).astype('float')

rng = np.random.RandomState(seed=11)
item_ids = list(responses['item id'].unique())
mab = LinearExample(rng=rng, arms=item_ids, l2_lambda=1, alpha=1, regression='ts', n_jobs=1, backend=None)
mab.arm_to_model[1]

In [5]:
scaler = pickle.load(open('scaler.pkl', 'rb'))
contexts = scaler.fit_transform(contexts)
test_contexts = scaler.transform(test_contexts)

mab.fit(decisions, rewards, contexts)
lin_expectations = mab.predict_expectations(test_contexts)

lin_expectations[0][1]

13.999455973138591

In [6]:
lin_expectations[0][2]

13.619175409413963

In [7]:
pickle.dump(mab, open('lin_mab3.pkl', 'wb'))
pickle.dump(lin_expectations, open('lin_expectations3.pkl', 'wb'))

# SageMaker

In [2]:
platform.platform()

'Linux-4.14.225-121.357.amzn1.x86_64-x86_64-with-glibc2.9'

In [3]:
print(np.__version__)

1.19.5


In [4]:
users = pd.read_csv('users.csv')
responses = pd.read_csv('responses.csv')

train = users[users['set']=='train']
test = users[users['set']=='test']

train = train.merge(responses, how='left', on='user id')
context_features = [c for c in users.columns if c not in ['user id', 'set']]

decisions = MAB._convert_array(train['item id'])
rewards = MAB._convert_array(train['rated'])
contexts = MAB._convert_matrix(train[context_features]).astype('float')

test_contexts = MAB._convert_matrix(test[context_features]).astype('float')

rng = np.random.RandomState(seed=11)
item_ids = list(responses['item id'].unique())
mab = LinearExample(rng=rng, arms=item_ids, l2_lambda=1, alpha=1, regression='ts', n_jobs=1, backend=None)
mab.arm_to_model[1]

In [5]:
scaler = pickle.load(open('scaler.pkl', 'rb'))
contexts = scaler.fit_transform(contexts)
test_contexts = scaler.transform(test_contexts)

mab.fit(decisions, rewards, contexts)
sgm_expectations = mab.predict_expectations(test_contexts)

sgm_expectations[0][1]

/home/ec2-user/anaconda3/envs/python3/lib/python3.6/site-packages/sklearn/base.py:334: UserWarning: Trying to unpickle estimator StandardScaler from version 0.23.2 when using version 0.23.0. This might lead to breaking code or invalid results. Use at your own risk.
  UserWarning)


13.999455973138849

In [6]:
sgm_expectations[0][2]

13.619175409413986

In [7]:
pickle.dump(mab, open('sgm_mab3.pkl', 'wb'))
pickle.dump(sgm_expectations, open('sgm_expectations3.pkl', 'wb'))